# IT362: Principles of Data Science

## Phase 1: Data Collection Research and Assessment  

### 1. Introduction
The **Natural Disaster Prediction Model: PREDINA** aims to analyze and predict the impacts of natural disasters based on historical data, with a focus on enhancing disaster preparedness and response strategies. The main research question guiding this project is: *How can historical data on natural disasters inform future predictions and improve community resilience?*

### 2. Importing Libraries
In this section, we will import the necessary libraries for our analysis and data processing.

In [54]:
# Install required libraries
!pip install openpyxl

# Import libraries
import pandas as pd
import requests
import csv
import json
import numpy as np


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


### 3. Datasets

- #### **EM-DAT: The International Disaster Database**
The EM-DAT Public Table is a global disaster database maintained by CRED, tracking natural and technological disasters. It includes data on fatalities, affected populations, and economic damages, and is used for research and disaster management.

In [55]:
# Read the Excel file
emdat = pd.read_excel("RawData/emdat.xlsx")

# Save the DataFrame to a CSV file
emdat.to_csv("Datasets/emdat.csv", index=False)

print("Conversion to CSV completed.")

Conversion to CSV completed.


In [56]:
emdat_df = pd.read_csv("Datasets/emdat.csv")
print(emdat_df.head())

          DisNo. Historic Classification Key Disaster Group Disaster Subgroup  \
0  1999-9388-DJI       No    nat-cli-dro-dro        Natural    Climatological   
1  1999-9388-SDN       No    nat-cli-dro-dro        Natural    Climatological   
2  1999-9388-SOM       No    nat-cli-dro-dro        Natural    Climatological   
3  2000-0001-AGO       No    tec-tra-roa-roa  Technological         Transport   
4  2000-0002-AGO       No    nat-hyd-flo-riv        Natural      Hydrological   

  Disaster Type Disaster Subtype External IDs Event Name  ISO  ...  \
0       Drought          Drought          NaN        NaN  DJI  ...   
1       Drought          Drought          NaN        NaN  SDN  ...   
2       Drought          Drought          NaN        NaN  SOM  ...   
3          Road             Road          NaN        NaN  AGO  ...   
4         Flood   Riverine flood          NaN        NaN  AGO  ...   

  Reconstruction Costs ('000 US$) Reconstruction Costs, Adjusted ('000 US$)  \
0            

- #### **Kaggle Dataset : ALL NATURAL DISASTERS 1900-2021 / EOSDIS**
This dataset, hosted on Kaggle, provides a record of natural disasters worldwide from 1900 to 2021, sourced from NASA's Earth Observing System Data and Information System (EOSDIS). It includes details such as disaster type, location, dates, and impacts (e.g., fatalities, affected populations, and economic damages), making it useful for analyzing historical disaster trends and impacts.

In [57]:
kaggle_df = pd.read_csv("RawData/EOSDIS.csv") #This line loads the CSV file into a pandas DataFrame called kaggle_df.
print(kaggle_df.head()) #This line prints the first five rows of the DataFrame, allowing us to inspect the format of the data.

   Year   Seq Glide Disaster Group Disaster Subgroup      Disaster Type  \
0  1900  9002   NaN        Natural    Climatological            Drought   
1  1900  9001   NaN        Natural    Climatological            Drought   
2  1902    12   NaN        Natural       Geophysical         Earthquake   
3  1902     3   NaN        Natural       Geophysical  Volcanic activity   
4  1902    10   NaN        Natural       Geophysical  Volcanic activity   

  Disaster Subtype Disaster Subsubtype   Event Name     Country  ...  \
0          Drought                 NaN          NaN  Cabo Verde  ...   
1          Drought                 NaN          NaN       India  ...   
2  Ground movement                 NaN          NaN   Guatemala  ...   
3         Ash fall                 NaN  Santa Maria   Guatemala  ...   
4         Ash fall                 NaN  Santa Maria   Guatemala  ...   

  No Affected No Homeless Total Affected Insured Damages ('000 US$)  \
0         NaN         NaN            NaN     

Here we read a CSV file named EOSDIS.csv located in the Datasets directory using the pandas library. We then display the first few rows of the dataframe to get an overview of the data structure and contents.

- ### **Global Disaster Alert and Coordination System (GDACS)**
The GDACS API provides real-time alerts on natural disasters such as earthquakes, tsunamis, and storms, offering data on disaster type, location, magnitude, and impact. It is useful for monitoring, coordinating disaster response efforts, and supplementing data for building predictive models.

In [58]:
def save_raw_dataset():
    LATEST_EVENTS_URL = 'https://www.gdacs.org/gdacsapi/api/events/geteventlist/EVENTS4APP'
    
    try:
        # Send a GET request to the API
        response = requests.get(LATEST_EVENTS_URL)
        response.raise_for_status()
        
        # Save the raw JSON data to a file
        with open('RawData/gdacs_dataset.json', 'w') as f:
            json.dump(response.json(), f)
        
        print("Raw dataset saved to 'gdacs_dataset.json'.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")

# Example usage
if __name__ == "__main__":
    save_raw_dataset()

Raw dataset saved to 'gdacs_dataset.json'.


In [59]:
def load_and_extract_columns():
    # Load the raw dataset from the saved file
    with open('RawData/gdacs_dataset.json', 'r') as f:
        dataset = json.load(f)
    
    # Extract specific columns from the dataset
    extracted_data = []
    for event in dataset.get('features', []):
        properties = event.get('properties', {})
        coordinates = event["geometry"]["coordinates"]
        latitude = coordinates[1]  # Lat from GeoJSON
        longitude = coordinates[0]  # Lon from GeoJSON

        # Extract magnitude and its unit directly from severitydata
        severity_data = properties.get("severitydata", {})
        magnitude = severity_data.get("severity", 0)
        magnitude_unit = severity_data.get("severityunit", "")  # Get the unit

        extracted_data.append({
            "Event Name": properties.get("eventname", "Unknown"),
            "Country": properties.get("country", "Unknown"),
            "ISO": properties.get("iso3", "Unknown"),
            "Disaster Group": properties.get("eventtype", "Unknown"),
            "Latitude": latitude,
            "Longitude": longitude,
            "Start Year": properties["fromdate"][:4],  # Extract year from date
            "Start Month": properties["fromdate"][5:7],  # Extract month
            "Start Day": properties["fromdate"][8:10],  # Extract day
            "End Year": properties["todate"][:4] if properties["todate"] else None,
            "End Month": properties["todate"][5:7] if properties["todate"] else None,
            "End Day": properties["todate"][8:10] if properties["todate"] else None,
            "Magnitude": magnitude,  # Directly include the magnitude value
            "Magnitude Unit": magnitude_unit,  # Include the magnitude unit
            "Losses": properties.get("economicloss", None)
        })
    
    return extracted_data

def save_to_csv(data, filename='RawData/gdacs_disasters.csv'):
    # Save the extracted data to a CSV file
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = data[0].keys()  # Get the column names from the first data entry
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()  # Write the header
        writer.writerows(data)  # Write the data rows

    print(f"Data saved to {filename}.")

# Example usage
if __name__ == "__main__":
    data = load_and_extract_columns()
    print(data)  # Print the extracted data
    save_to_csv(data)  # Save the extracted data as a CSV file

[{'Event Name': '', 'Country': 'Indonesia', 'ISO': 'IDN', 'Disaster Group': 'EQ', 'Latitude': -1.9708, 'Longitude': 138.9033, 'Start Year': '2025', 'Start Month': '02', 'Start Day': '07', 'End Year': '2025', 'End Month': '02', 'End Day': '07', 'Magnitude': 5.0, 'Magnitude Unit': 'M', 'Losses': None}, {'Event Name': 'TALIAH-25', 'Country': '', 'ISO': '', 'Disaster Group': 'TC', 'Latitude': -15.8, 'Longitude': 96.9, 'Start Year': '2025', 'Start Month': '02', 'Start Day': '02', 'End Year': '2025', 'End Month': '02', 'End Day': '07', 'Magnitude': 157.4064, 'Magnitude Unit': 'km/h', 'Losses': None}, {'Event Name': 'VINCE-25', 'Country': 'Cocos (Keeling) Islands', 'ISO': '', 'Disaster Group': 'TC', 'Latitude': -20.0, 'Longitude': 75.83, 'Start Year': '2025', 'Start Month': '02', 'Start Day': '01', 'End Year': '2025', 'End Month': '02', 'End Day': '07', 'Magnitude': 253.331712, 'Magnitude Unit': 'km/h', 'Losses': None}, {'Event Name': '', 'Country': 'Taiwan', 'ISO': 'TWN', 'Disaster Group': '

In [60]:
# Load the dataset
df = pd.read_csv('RawData/gdacs_disasters.csv')

# 1. Rename the 'Disaster Group' column and change its values
df.rename(columns={'Disaster Group': 'disaster type'}, inplace=True)

# Create a mapping for the disaster types
disaster_type_mapping = {
    'TC': 'Storm',
    'EQ': 'Earthquake',
    'FL': 'Flood',
    'VO': 'Volcanic Activity',
    'DR': 'Drought',
    'WF': 'Wildfire'
}

# Map the disaster types
df['disaster type'] = df['disaster type'].map(disaster_type_mapping).fillna(df['disaster type'])

# 2. Add new columns with NaN values
new_columns = ['Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless', 'Total Affected', 'Total Damages (\'000 US$)']
for column in new_columns:
    df[column] = np.nan

# 3. Add a column called 'Dis. Group' with all values set to 'Natural'
df['Dis. Group'] = 'Natural'

# 4. Map the disaster subgroup directly based on the new disaster type values
disaster_subgroup_mapping = {
    'Storm': 'Meteorological',
    'Earthquake': 'Geophysical',
    'Flood': 'Hydrological',
    'Volcanic Activity': 'Geophysical',
    'Drought': 'Climatological',
    'Wildfire': 'Climatological'
}

df['Disaster Subgroup'] = df['disaster type'].map(disaster_subgroup_mapping).fillna(np.nan)

# Save the modified DataFrame back to CSV (optional)
df.to_csv('Datasets/modified_gdacs_disasters.csv', index=False)

print(df.head())  # Display the first few rows of the modified DataFrame

  Event Name                  Country  ISO disaster type  Latitude  Longitude  \
0        NaN                Indonesia  IDN    Earthquake   -1.9708   138.9033   
1  TALIAH-25                      NaN  NaN         Storm  -15.8000    96.9000   
2   VINCE-25  Cocos (Keeling) Islands  NaN         Storm  -20.0000    75.8300   
3        NaN                   Taiwan  TWN    Earthquake   23.1911   120.5429   
4        NaN                      NaN  NaN    Earthquake    3.6920   127.8168   

   Start Year  Start Month  Start Day  End Year  ...  Magnitude Unit  Losses  \
0        2025            2          7      2025  ...               M     NaN   
1        2025            2          2      2025  ...            km/h     NaN   
2        2025            2          1      2025  ...            km/h     NaN   
3        2025            2          7      2025  ...               M     NaN   
4        2025            2          7      2025  ...               M     NaN   

   Total Deaths No. Injured  No.

Convert Data Types to Match

In [61]:
numeric_columns = ["Latitude", "Longitude", "Start Year", "Start Month", "Start Day", 
                   "End Year", "End Month", "End Day", "Total Deaths", "No. Injured", 
                   "No. Affected", "No. Homeless", "Total Affected", "Total Damages ('000 US$)", "CPI", "Magnitude"]

for col in numeric_columns:
    if col in emdat_df.columns:
        emdat_df[col] = pd.to_numeric(emdat_df[col], errors='coerce')  # Convert to numeric, set errors to NaN
    if col in kaggle_df.columns:
        kaggle_df[col] = pd.to_numeric(kaggle_df[col], errors='coerce')

In [62]:
string_columns = ["Year", "Disaster Group", "Disaster Subgroup", "Disaster Type", "ISO", "Magnitude Scale"]

for col in string_columns:
    if col in emdat_df.columns:
        emdat_df[col] = emdat_df[col].astype(str).str.strip()  # Ensure string format
    if col in kaggle_df.columns:
        kaggle_df[col] = kaggle_df[col].astype(str).str.strip()

print("Updated df1 column types:\n", emdat_df.dtypes)
print("Updated df2 column types:\n",kaggle_df.dtypes)

Updated df1 column types:
 DisNo.                                        object
Historic                                      object
Classification Key                            object
Disaster Group                                object
Disaster Subgroup                             object
Disaster Type                                 object
Disaster Subtype                              object
External IDs                                  object
Event Name                                    object
ISO                                           object
Country                                       object
Subregion                                     object
Region                                        object
Location                                      object
Origin                                        object
Associated Types                              object
OFDA/BHA Response                             object
Appeal                                        object
Declaration        

### 4. Data Intergration

In [63]:
# Load the datasets
gdacs_df = pd.read_csv('Datasets/modified_gdacs_disasters.csv')
eosdis_df = pd.read_csv('RawData/EOSDIS.csv')
emdat_df = pd.read_csv('RawData/emdat.csv')

# Print the columns of each DataFrame to debug
print("GDACS columns:", gdacs_df.columns.tolist())
print("EOSDIS columns:", eosdis_df.columns.tolist())
print("EMDAT columns:", emdat_df.columns.tolist())

# Create a mapping of the columns to standardize names
column_mapping_gdacs = {
    'Dis. Group': 'Disaster Group',
    'Disaster Subgroup': 'Disaster Subgroup',
    'disaster type': 'Disaster Type',
    'ISO': 'ISO',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    'Start Year': 'Start Year',
    'Start Month': 'Start Month',
    'Start Day': 'Start Day',
    'End Year': 'End Year',
    'End Month': 'End Month',
    'End Day': 'End Day',
    'Total Deaths': 'Total Deaths',
    'No. Injured': 'No. Injured',
    'No. Affected': 'No. Affected',
    'No. Homeless': 'No. Homeless',
    'Total Affected': 'Total Affected',
    "Total Damages ('000 US$)": "Total Damages ('000 US$)",
    'Magnitude': 'Magnitude',
    'Magnitude Unit': 'Magnitude Scale'
}

column_mapping_eosdis = {
    'Disaster Group': 'Disaster Group',
    'Disaster Subgroup': 'Disaster Subgroup',
    'Disaster Type': 'Disaster Type',
    'ISO': 'ISO',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    'Start Year': 'Start Year',
    'Start Month': 'Start Month',
    'Start Day': 'Start Day',
    'End Year': 'End Year',
    'End Month': 'End Month',
    'End Day': 'End Day',
    'Total Deaths': 'Total Deaths',
    'No Injured': 'No. Injured',
    'No Affected': 'No. Affected',
    'No Homeless': 'No. Homeless',
    'Total Affected': 'Total Affected',
    "Total Damages ('000 US$)": "Total Damages ('000 US$)",
    'Dis Mag Value': 'Magnitude',
    'Dis Mag Scale': 'Magnitude Scale'
}

column_mapping_emdat = {
    'Disaster Group': 'Disaster Group',
    'Disaster Subgroup': 'Disaster Subgroup',
    'Disaster Type': 'Disaster Type',
    'ISO': 'ISO',
    'Latitude': 'Latitude',
    'Longitude': 'Longitude',
    'Start Year': 'Start Year',
    'Start Month': 'Start Month',
    'Start Day': 'Start Day',
    'End Year': 'End Year',
    'End Month': 'End Month',
    'End Day': 'End Day',
    'Total Deaths': 'Total Deaths',
    'No. Injured': 'No. Injured',
    'No. Affected': 'No. Affected',
    'No. Homeless': 'No. Homeless',
    'Total Affected': 'Total Affected',
    "Total Damages ('000 US$)": "Total Damages ('000 US$)",
    'Magnitude': 'Magnitude',
    'Magnitude Scale': 'Magnitude Scale'
}

# Rename columns
gdacs_df.rename(columns=column_mapping_gdacs, inplace=True)
eosdis_df.rename(columns=column_mapping_eosdis, inplace=True)
emdat_df.rename(columns=column_mapping_emdat, inplace=True)

# Check available columns after renaming
print("GDACS columns after rename:", gdacs_df.columns.tolist())
print("EOSDIS columns after rename:", eosdis_df.columns.tolist())
print("EMDAT columns after rename:", emdat_df.columns.tolist())

# Keep only the relevant columns from each DataFrame
columns_to_keep = ['Disaster Group', 'Disaster Subgroup', 'Disaster Type', 'ISO',
                   'Latitude', 'Longitude', 'Start Year', 'Start Month',
                   'Start Day', 'End Year', 'End Month', 'End Day',
                   'Total Deaths', 'No. Injured', 'No. Affected',
                   'No. Homeless', 'Total Affected', "Total Damages ('000 US$)",
                   'Magnitude', 'Magnitude Scale']

# Filter DataFrames based on available columns
gdacs_df = gdacs_df[columns_to_keep]
eosdis_df = eosdis_df[columns_to_keep]

# Check if 'Total Damages (\'000 US$)' exists in emdat_df
if "Total Damages ('000 US$)" in emdat_df.columns:
    emdat_df = emdat_df[columns_to_keep]
else:
    # Adjust columns_to_keep to remove the missing column for emdat_df
    columns_to_keep_emdat = [col for col in columns_to_keep if col in emdat_df.columns]
    emdat_df = emdat_df[columns_to_keep_emdat]

# Reset index to ensure unique indices
gdacs_df.reset_index(drop=True, inplace=True)
eosdis_df.reset_index(drop=True, inplace=True)
emdat_df.reset_index(drop=True, inplace=True)

# Check for duplicate rows
print("GDACS duplicates:", gdacs_df.duplicated().sum())
print("EOSDIS duplicates:", eosdis_df.duplicated().sum())
print("EMDAT duplicates:", emdat_df.duplicated().sum())

# If duplicates exist, you can drop them
gdacs_df.drop_duplicates(inplace=True)
eosdis_df.drop_duplicates(inplace=True)
emdat_df.drop_duplicates(inplace=True)

# Concatenate the DataFrames
merged_df = pd.concat([gdacs_df, eosdis_df, emdat_df], ignore_index=True)

# Save the merged dataset
merged_df.to_csv('Datasets/integrated.csv', index=False)

# Print final shape of the merged DataFrame
print("Merged DataFrame shape:", merged_df.shape)

GDACS columns: ['Event Name', 'Country', 'ISO', 'disaster type', 'Latitude', 'Longitude', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 'Magnitude', 'Magnitude Unit', 'Losses', 'Total Deaths', 'No. Injured', 'No. Affected', 'No. Homeless', 'Total Affected', "Total Damages ('000 US$)", 'Dis. Group', 'Disaster Subgroup']
EOSDIS columns: ['Year', 'Seq', 'Glide', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type', 'Disaster Subtype', 'Disaster Subsubtype', 'Event Name', 'Country', 'ISO', 'Region', 'Continent', 'Location', 'Origin', 'Associated Dis', 'Associated Dis2', 'OFDA Response', 'Appeal', 'Declaration', 'Aid Contribution', 'Dis Mag Value', 'Dis Mag Scale', 'Latitude', 'Longitude', 'Local Time', 'River Basin', 'Start Year', 'Start Month', 'Start Day', 'End Year', 'End Month', 'End Day', 'Total Deaths', 'No Injured', 'No Affected', 'No Homeless', 'Total Affected', "Insured Damages ('000 US$)", "Total Damages ('000 US$)", 'CPI', 'Adm Level', 'Admin1 C